# VISIULIZATION OF DJI WITH RSI,BOLLINGER BANDBY USING BOKEH SERVER

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
import ta
from pandas_datareader import data
from bokeh.io import output_file, show,curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool,ColumnDataSource,Slider
from bokeh.layouts import row,column
from bokeh.models.formatters import DatetimeTickFormatter
import warnings
warnings.filterwarnings('ignore')

In [2]:
stock = '^DJI'
start_date = '2019-07-01'
end_date = '2020-07-01'

df = data.DataReader(stock,'yahoo',start_date,end_date)
df['Date'] =df.index

In [3]:

N_rsi=11
N_bb = 20
N_mva = 20



source = ColumnDataSource(data={'x':df['Date'],'y0': df['Adj Close'],'y1':df['Open'],'y2':df['High'],
                                'y3':df['Low'],'y4':df['Volume'],'y5':df['Adj Close'].rolling(N_mva).mean(),'y6':ta.momentum.rsi(close=df['Adj Close'],n=N_rsi),
                                'y7':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_mavg(),
                          'y8':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_hband(),
                                'y9':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_lband()})

source1 = ColumnDataSource(data={'x':df['Date'],'y0': df['Adj Close'],'y1':df['Open'],'y2':df['High'],
                                'y3':df['Low'],'y4':df['Volume'],'y5':df['Adj Close'].rolling(N_mva).mean(),'y6':ta.momentum.rsi(close=df['Adj Close'],n=N_rsi),
                                'y7':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_mavg(),
                          'y8':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_hband(),
                                'y9':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_lband()})




In [4]:
# Create plots and widgets
fig1 = figure(title=stock+' Price & Technical Analysis',x_axis_type='datetime',x_axis_label='Date',y_axis_label='$',plot_width=1000, plot_height=700)
plot1 = fig1.line('x','y0',source=source,legend_label='Adjusted Close')
fig1.add_tools(HoverTool(renderers=[plot1],tooltips=[('Open','@y1'),('High','@y2'),('Low','@y3'),('Adjusted Close',
                        '@y0'),('Volume', '@y4'),('Date','@x{%F}')],formatters={'@x': 'datetime'}))
plot2 = fig1.line('x','y7',source=source,color='Brown',legend_label='moving Average')
fig1.add_tools(HoverTool(renderers=[plot2],tooltips=[('Moving Average','@y7')]))


plot3 = fig1.line('x','y8',source=source,color='Black',legend_label='Bolinger Band')
fig1.add_tools(HoverTool(renderers=[plot3],tooltips=[('Bolinger Band','@y8')]))

plot4 = fig1.line('x','y9',source=source,color='Black',legend_label='Bolinger Band')
fig1.add_tools(HoverTool(renderers=[plot4],tooltips=[('Bolinger Band','@y9')]))

fig1.legend.location = 'top_left'



fig2 = figure(title=stock+' RSI',x_axis_type='datetime',x_axis_label='Date',y_axis_label='%',plot_width=1000, plot_height=700)

plot5 = fig2.line('x','y6',source=source1,color='red',legend_label='RSI')
fig2.add_tools(HoverTool(renderers=[plot5],tooltips=[('RSI','@y6')]))
fig2.legend.location = 'top_left'





In [5]:
# Add callback to widgets


slider_rsi = Slider(start=1, end=200, value=N_rsi,step=1, title='Number of Days for Rolling(RSI)')
slider_bb = Slider(start=1, end=50, value=N_bb,step=1, title='Number of Days for Rolling(Bolinger Band)')

def callback(attr, old, new):
    N_rsi = slider_rsi.value
    source1.data={'x':df['Date'],'y0': df['Adj Close'],'y1':df['Open'],'y2':df['High'],
                                'y3':df['Low'],'y4':df['Volume'],'y5':df['Adj Close'].rolling(N_mva).mean(),'y6':ta.momentum.rsi(close=df['Adj Close'],n=N_rsi),
                                'y7':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_mavg(),
                          'y8':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_hband(),
                                'y9':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_lband()}

def callback1(attr, old, new):
    N_bb = slider_bb.value
    source.data={'x':df['Date'],'y0': df['Adj Close'],'y1':df['Open'],'y2':df['High'],
                                'y3':df['Low'],'y4':df['Volume'],'y5':df['Adj Close'].rolling(N_mva).mean(),'y6':ta.momentum.rsi(close=df['Adj Close'],n=N_rsi),
                                'y7':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_mavg(),
                          'y8':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_hband(),
                                'y9':(ta.volatility.BollingerBands(close=df["Adj Close"], n=N_bb, ndev=2)).bollinger_lband()}






slider_rsi.on_change('value', callback)
slider_bb.on_change('value', callback1)






c = column(children =[fig1, fig2], sizing_mode = 'stretch_both')





In [6]:
# Arrange plots and widgets in layouts

layout = column(slider_rsi,slider_bb , c)


curdoc().add_root(layout)

In [ ]:
! bokeh serve --show bokeh_dji.ipynb